In [ ]:
import pandas as pd
import numpy as np
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from gensim.models import Word2Vec
model = Word2Vec.load('/content/drive/MyDrive/Applied Deep Learning Poli Sci/Foley and Dorner Repo/Models/w2v100_tweets_model.model')

word_vectors = model.wv.vectors

In [ ]:
import json

# Load the JSON file into a dictionary
with open('/content/drive/MyDrive/Applied Deep Learning Poli Sci/Foley and Dorner Repo/Models/word_to_cluster_model.json', 'r') as file:
    word_to_cluster = json.load(file)

# Convert the dictionary values to np.int32
word_to_cluster = {word: np.int32(cluster) for word, cluster in word_to_cluster.items()}

In [ ]:
final_merged = pd.read_csv('/content/drive/MyDrive/Applied Deep Learning Poli Sci/Foley and Dorner Repo/Data/final_merged.csv')
user_data = pd.read_csv('/content/drive/MyDrive/Applied Deep Learning Poli Sci/Foley and Dorner Repo/Data/user_data.csv')
bin_user = pd.read_csv('/content/drive/MyDrive/Applied Deep Learning Poli Sci/Foley and Dorner Repo/Data/bin_user.csv')

#final_merged_int = pd.merge(final_merged, user_data, on='screen_name', how='left')
#final_merged2 = pd.merge(final_merged_int, bin_user, on='screen_name', how='left')

In [ ]:
# Load the model
clf = joblib.load('/content/drive/MyDrive/Applied Deep Learning Poli Sci/Foley and Dorner Repo/Models/logistic_regression_model.pkl')

# Extract the coefficients
coefficients = clf.coef_

# Assuming binary classification
cluster_importance = np.abs(coefficients).mean(axis=0)  # Taking mean of absolute values across classes

# Normalize the importance values for better visualization
cluster_importance = cluster_importance / cluster_importance.max()

# Get the indices of the top N most predictive clusters
top_n = 10
top_clusters_indices = np.argsort(cluster_importance)[-top_n:][::-1]

# Print the indices of the top clusters
print(f"Indices of the top {top_n} most predictive clusters: {top_clusters_indices}")

# Assuming 'word_to_cluster' is a dictionary that maps words to their respective clusters
# and 'model.wv.index_to_key' contains the list of words in the Word2Vec model

# Create a dictionary to store words for each top cluster
top_clusters_words = {index: [] for index in top_clusters_indices}

# Populate the dictionary with words
for word, cluster in word_to_cluster.items():
    if cluster in top_clusters_indices:
        top_clusters_words[cluster].append(word)

# Display the words in each of the top clusters
for cluster, words in top_clusters_words.items():
    print(f"Cluster {cluster}:")
    print(", ".join(words[:20]))  # Displaying only the first 20 words for brevity
    print()

# Create a bar plot with labels for the top clusters
plt.figure(figsize=(15, 7))
bar_colors = ['red' if i in top_clusters_indices else 'blue' for i in range(len(cluster_importance))]
sns.barplot(x=np.arange(len(cluster_importance)), y=cluster_importance, palette=bar_colors)

# Add labels for the top clusters
for i in top_clusters_indices:
    plt.text(i, cluster_importance[i], f'{i}', ha='center', va='bottom', fontsize=10, color='black')

plt.xlabel('Cluster Index')
plt.ylabel('Importance')
plt.title('Importance of Each Word2Vec Cluster in Predicting Ideologies')
plt.show()